In [10]:
import os
import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import math
import numpy as np
import itertools
import collections
from datetime import datetime
import pickle

In [2]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [3]:
# get all segment file names and save in Filenames
def scan(folder):        
    x=[]
    for path,dirs,files in os.walk(folder):
        for f in files:
            if f.startswith('segment'):
                x.append(os.path.join(path,f))
    return(x)

folder = './waymo-od/data'

Filenames=scan(folder)

In [4]:
def getID(f):         #get the ID of segment from file name
    f = f.replace('_with_camera_labels.tfrecord', '')
    f = f.split('segment-',1)[1]
    return f

## Each frame includes:
#### 1. ***context***: similar for all the frames in one segment
#### 2. ***timestamp_micros*** : should be devided to 1000000
#### 3. ***pose***
#### 4. ***laser_labels*** : includes
#### 5. ***camera_labels***

In [5]:
def CountLLabels(frame):    #number of laser labels in a frame
    return len(frame.laser_labels)

def converttime(t):
    t = t/1000000
    t = datetime.fromtimestamp(t)
    return t

In [6]:
def Ped2df(SegID, tstamp, pedLoc, 
           pedMeta,VehPos,tday,w):
    
    length = len(pedLoc)
    t,px, py, pz,\
    px_GF,py_GF,pz_GF,\
    vx_GF,vy_GF,vz_GF,\
    pw, pl, pht,\
    phd, psx, psy,\
    paccx, paccy = ([] for i in range(18))
    
    for i in range(length):
        t.append(tstamp[i])
        pw.append(pedLoc[i].width)       
        pl.append(pedLoc[i].length)
        pht.append(pedLoc[i].height)
        phd.append(pedLoc[i].heading)
        psx.append(pedMeta[i].speed_x)
        psy.append(pedMeta[i].speed_y)
        paccx.append(pedMeta[i].accel_x)
        paccy.append(pedMeta[i].accel_y)

        # Ped position in Vehicle Frame
        PedPoseVF = np.array([pedLoc[i].center_x,
                              pedLoc[i].center_y,
                              pedLoc[i].center_z,1])
        px.append(PedPoseVF[0])
        py.append(PedPoseVF[1])
        pz.append(PedPoseVF[2])
        # Ped position in Global Frame
        PedPoseGF = np.matmul(np.reshape(np.array(VehPos[i].transform),[4,4]),PedPoseVF)
        px_GF.append(PedPoseGF[0])
        py_GF.append(PedPoseGF[1])
        pz_GF.append(PedPoseGF[2])
        # SDC position in Global Frame
        VehPoseGF = np.matmul(np.reshape(np.array(VehPos[i].transform),[4,4]),[0,0,0,1])
        vx_GF.append(VehPoseGF[0])
        vy_GF.append(VehPoseGF[1])
        vz_GF.append(VehPoseGF[2])
    
    data = np.array([t, px, py, pz,
                     px_GF, py_GF, pz_GF,
                     vx_GF, vy_GF, vz_GF,
                     pw, pl, pht, phd,
                     psx, psy, paccx, paccy]).T.tolist()
    
    columnsnames= ['Timestamp', 'x', 'y', 'z',
                   'x_GF', 'y_GF', 'pz_GF',
                    'x_GF', 'y_GF', 'z_GF',
                   'Width', 'Length','Height','Heading',
                   'Speed x','Speed y', 'accel x', 'accel y']
    pedDF = pd.DataFrame(data, columns = columnsnames)
    
    #transform = pedDF['vpose'].apply(pd.Series)
    #transform = transform.rename(columns= lambda x: 'AVtransform_' +str(x))
    #pedDF = pedDF.drop(columns= 'vpose')
    #pedDF = pd.concat([pedDF[:],transform[:]],axis=1)
    
    pedDF['Weather'] = w
    pedDF['Time of Day'] = tday
    pedDF['Segment ID'] = SegID
    return pedDF
 

In [7]:
def PedExtract(SegID):
    pedLoc = collections.defaultdict(list)                             #pedestrians label box
    pedMeta = collections.defaultdict(list)                            #pedestrians label metadata
    VehPos =  collections.defaultdict(list)                            #SDC transformation matrix
    tstamp = collections.defaultdict(list) 
    tday = Frames[SegID][0].context.stats.time_of_day
    w = Frames[SegID][0].context.stats.weather
    
    for j in range(len(Frames[SegID])):
        t = Frames[SegID][j].timestamp_micros                      #timestamp
        vpos = Frames[SegID][j].pose                                
        Nlabels = CountLLabels(Frames[SegID][j])                   # number of laser labels
       
        for k in range(Nlabels):
            LidarLab = Frames[SegID][j].laser_labels[k]
            if  LidarLab.type == 2:                      # type 2: pedestrian
                
                PedID = LidarLab.id

                tstamp[PedID].append(t)
                pedLoc[PedID].append(LidarLab.box)

                
                pedMeta[PedID].append(LidarLab.metadata)
                
                
                VehPos[PedID].append(Frames[SegID][j].pose)
    
    Ped = {}
    for p in list(pedLoc.keys()):
        Ped[p] = Ped2df(SegID, tstamp[p], pedLoc[p],
                        pedMeta[p],VehPos[p],tday, w)
        
    
    return Ped      

In [8]:
def MultiFrame(Frames):
    PedTraj = {}
    for i in range(len(Frames)):
        SegID = list(Frames.keys())[i]
        PedTraj.update(PedExtract(SegID))
    return PedTraj
    

In [16]:
### Extract segment files in Frames, each file consists of multiple frames

for i in range(2):
    Frames = {}
    for file in Filenames[3*i:(3*i+3)]:
        SegID = getID(file)
        Segment = []
        dataset = tf.data.TFRecordDataset(file, compression_type='')
        for data in dataset:
            frame = open_dataset.Frame()
            frame.ParseFromString(bytearray(data.numpy()))
            Segment.append(frame)
        Frames[SegID] = Segment
        
    PedData = MultiFrame(Frames)
    name = f"PedTables/Iterate:{3*i}_{3*i+3}.p"
    with open(name, 'wb') as fp:
        pickle.dump(PedData, fp)
    del PedData
    del Frames

In [ ]:
## Open later
with open('PedTables/Iterate:0_3.p', 'rb') as fp:
    PedData = pickle.load(fp)
#PedIDs = list(PedData.keys())

In [17]:
PedData

NameError: name 'PedData' is not defined